In [ ]:
import os
import json

import pandas as pd
from tqdm import tqdm


In [ ]:
config_json="config/config.json"
station_json="stations/stations.json"
hypoinverse_station="hypoinv/stations.dat"
hypodd_station="hypodd/stations.dat"
data_path="./"

In [ ]:


with open(config_json, "r") as fp:
    config = json.load(fp)

hypodd_path = os.path.join(data_path, "hypodd")
if not os.path.exists(hypodd_path):
    os.mkdir(hypodd_path)
hypoinv_path = os.path.join(data_path, "hypoinv")
if not os.path.exists(hypoinv_path):
    os.mkdir(hypoinv_path)

stations = pd.read_json(station_json, orient="index")

converted_hypoinverse = []
converted_hypodd = {}

for sta, row in tqdm(stations.iterrows()):

    network_code, station_code, comp_code, channel_code = sta.split(".")
    station_weight = " "
    lat_degree = int(row["latitude"])
    lat_minute = (row["latitude"] - lat_degree) * 60
    north = "N" if lat_degree >= 0 else "S"
    lng_degree = int(row["longitude"])
    lng_minute = (row["longitude"] - lng_degree) * 60
    west = "W" if lng_degree <= 0 else "E"
    elevation = row["elevation(m)"]
    line_hypoinverse = f"{station_code:<5} {network_code:<2} {comp_code[:-1]:<1}{channel_code:<3} {station_weight}{abs(lat_degree):2.0f} {abs(lat_minute):7.4f}{north}{abs(lng_degree):3.0f} {abs(lng_minute):7.4f}{west}{elevation:4.0f}\n"
    converted_hypoinverse.append(line_hypoinverse)

    tmp_code = f"{station_code}{channel_code}"
    converted_hypodd[
        f"{station_code}{channel_code}"
    ] = f"{tmp_code:<8s} {row['latitude']:.3f} {row['longitude']:.3f}\n"

with open(hypoinverse_station, "w") as f:
    f.writelines(converted_hypoinverse)

with open(hypodd_station, "w") as f:
    for k, v in converted_hypodd.items():
        f.write(v)

In [ ]:
def split_hypodd(
    config_json,
    catalog_csv,
) -> list:

    import json
    import pandas as pd

    with open(config_json, "r") as fp:
        config = json.load(fp)

    events = pd.read_csv(catalog_csv)

    if "MAXEVENT" in config["hypodd"]:
        MAXEVENT = config["hypodd"]["MAXEVENT"]
    else:
        MAXEVENT = 1e4  ## segment by time

    MAXEVENT = len(events) // ((len(events) - 1) // MAXEVENT + 1) + 1
    num_parallel = int((len(events) - 1) // MAXEVENT + 1)

    return list(range(num_parallel))

In [ ]:
nodes = split_hypodd(
    "config/config.json",
    "gamma/gamma_catalog.csv",
)
with open('num_parallel.txt', 'w') as f:
    f.write(str(nodes))

In [8]:
# Kubeflow Pipelines UI用のメタデータ出力
if os.environ.get('ELYRA_RUNTIME_ENV') == 'kfp':
    # For information about Elyra environment variables refer to
    # https://elyra.readthedocs.io/en/stable/user_guide/best-practices-file-based-nodes.html#proprietary-environment-variables

    metadata = {
        'outputs': [
            {
                'storage': 'inline',
                'source': f'# Convert Station Format Complete\n...',
                'type': 'markdown',
            }
        ]
    }

    with open('mlpipeline-ui-metadata.json', 'w') as f:
        json.dump(metadata, f)